In [18]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as img
import numpy as np
import cv2
import os, shutil
from scipy.spatial import distance
import xml.etree.ElementTree as ET

### Annotation 파일 읽어오기
https://codechacha.com/ko/python-parsing-xml-and-pretty-print/

In [19]:
import os 

path = './empty'
annotation_list = os.listdir(path) 
annotation_list.remove('images')
print(annotation_list)

['annotations.xml']


### 함수 클래스 정의

In [20]:
class Utils:
    def __init__(self):
        pass
    
    # 각 행의 n 번째 열을 기준으로 sort (오름차순)
    # x : nd array
    # axis : 기준 열
    def sort_ndarray_by_col(self, x, axis=0):
        size = x.shape[0]
        col = [x[i][axis] for i in range(size)] # 기준 행의 원소들을 추출
        sorted_col = sorted(col) # 추출된 원소들을 정렬
        rank = [sorted_col.index(x[i][axis]) for i in range(size)] # 정렬된 index 추출
        #print(rank)
        try:
            sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬
        except: # 똑같은 순위가 있는 경우 예외처리
            overlap_rank = []
            for i in range(size):
                if rank.count(i) > 1:
                    overlap_rank.append(i)
            for r in overlap_rank:
                overlap_list = [i for i in range(size) if rank[i]==r]
                for k, idx in enumerate(overlap_list):
                    rank[idx] += k
            sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬

        return np.array(sorted_array)

    # 함수화
    def perspective_transform(self, src, point, height=256, width=256):    
        # 4 꼭지점 분류 
        sorted_by_y= self.sort_ndarray_by_col(point, 1)
        top_two = sorted_by_y[0:2]
        topLeft, topRight = self.sort_ndarray_by_col(top_two, 0)
        bottom_two = sorted_by_y[2:4]
        bottomLeft, bottomRight = self.sort_ndarray_by_col(bottom_two, 0)
        #print("tl:", topLeft)
        #print("bl", bottomLeft)
        #print("tr", topRight)
        #print("br", bottomRight)


        # 변환 전 4개 좌표 
        srcPoint = np.float32([topLeft, topRight, bottomRight , bottomLeft])

        # 변환 후 4개 좌표
        dstPoint = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

        # Perspective transformation
        matrix = cv2.getPerspectiveTransform(srcPoint, dstPoint)
        dst = cv2.warpPerspective(src, matrix, (width, height))

        return dst

    # 이미지 상에 polygon 그리기
    def draw_label(self, img_path, points, save_path=""):
        '''
        image = img.imread(img_path)
        plt.imshow(image)

        ax = plt.gca()

        point = np.array([[data['p1'][0], data['p1'][1]],
                          [data['p2'][0], data['p2'][1]],
                          [data['p3'][0], data['p3'][1]],
                          [data['p4'][0], data['p4'][1]]])
        #print(point)

        polygon = patches.Polygon(point, fill=None ,edgecolor='k',ls='solid',lw=3)

        ax.add_patch(polygon)

        plt.show()
        '''
        src = cv2.imread(img_path, cv2.IMREAD_COLOR)
        dst = cv2.polylines(src, np.int32([points]), True, (255, 0, 0), 6)

        return dst

    def extract_bb_points(self, points, width, height):
        if len(points.split(';')) == 4: # 라벨링된 point가 4개인 경우 = polygon 라벨링 (없음)
            ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                 (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1])),
                 (float(points.split(';')[2].split(',')[0]), float(points.split(';')[2].split(',')[1])),
                 (float(points.split(';')[3].split(',')[0]), float(points.split(';')[3].split(',')[1]))]

            # 4 꼭지점 분류 
            sorted_by_y= self.sort_ndarray_by_col(np.array(ps), 1)
            top_two = sorted_by_y[0:2]
            topLeft, topRight = self.sort_ndarray_by_col(top_two, 0)
            bottom_two = sorted_by_y[2:4]
            bottomLeft, bottomRight = list(self.sort_ndarray_by_col(bottom_two, 0))

        elif len(points.split(';')) == 2: # 라벨링된 point가 2개인 경우 = 직선 라벨링 (있음))
            ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                 (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1]))]

            # 2 꼭지점 분류
            if ps[0][0] < ps[1][0]:
                bottomLeft = ps[0]
                bottomRight = ps[1]
            else:
                bottomLeft = ps[1]
                bottomRight = ps[0]
        else:
            return None, 0, None

        print(len(points.split(';')))
        print('bb', bottomLeft, bottomRight) # bounding box 생성에 기준이 되는 주차면 앞 라인의 두 점

        # 라인으로부터 바운딩 박스 생성
        x_offset = 0
        width = distance.euclidean(bottomLeft, bottomRight)+x_offset
        y_offset = width/10.0
        height = -width*2/3.0
        #print(ps, width, height)


        b1 = (bottomLeft[0], bottomLeft[1]+y_offset)
        b2 = (bottomRight[0], bottomRight[1]+y_offset)
        b3 = (b2[0], b2[1]+height)
        b4 = (b1[0], b1[1]+height)
        print('ps:',np.array([b1, b2, b3, b4]), width, np.array(ps))

        return np.array([b1, b2, b3, b4]), width, np.array(ps)

    def get_occupied_info(self, polyline, width, height):
        label = polyline.get('label')
        points = polyline.get('points') # 2개
        bb_points, width, _ = self.extract_bb_points(points, width, height)
        return label, width, bb_points

    def get_empty_info(self, polygon, width, height):
        label = polygon.get('label')
        points = polygon.get('points') # 4개 or 5개
        bb_points, width, poly_points = self.extract_bb_points(points, width, height)
        empty_type_dict = {'일반형':0, '경형':1, '장애인전용':2, '여성우선주차장':3, '환경친화적 자동차 전용':4, 'etc':5, '기타':5}
        empty_type = empty_type_dict[polygon.find('attribute').text]
        return label, width, bb_points, poly_points, empty_type

    def get_obstacle_info(self, polygon, width, height):
        label = polygon.get('label')
        points = polygon.get('points')
        bb_points, width, poly_points = self.extract_bb_points(points, width, height)
        obstacle_type_dict = {'사람':0, '이륜차':1, '기타':2}
        obs_type = obstacle_type_dict[polygon.find('attribute').text]
        return label, width, bb_points, poly_points, obs_type

In [21]:
#from utils import Utils
# 함수 클래스 인스턴스화
ut = Utils()

### 데이터셋 리스트 생성

In [22]:
dataset_list = []
for annotation in annotation_list:
    # annotation 파일 불러오기
    filePath = os.path.join(path, annotation)
    tree = ET.parse(filePath)
    root = tree.getroot()
    dataset_name = root.find("meta").find("task").find("name").text
    dataset_list.append(dataset_name)
    print(annotation, dataset_name)

annotations.xml Extend_Empty


### xml 파일로부터 라벨링 정보 추출

In [23]:
occupied_cnt, empty_cnt, obstacle_cnt = 0, 0, 0

datasets = {}
datasets['occupied'] = {}
datasets['empty'] = {}
datasets['obstacle'] = {}

for annotation in annotation_list:
    print(annotation)
    # annotation 파일 불러오기
    filePath = os.path.join(path, annotation)
    tree = ET.parse(filePath)
    root = tree.getroot()
    dataset_name = root.find("meta").find("task").find("name").text
    
    for image in root.findall("image"):
        image_attrib = image.attrib
        image_frame = image_attrib.get('id')
        image_name = image_attrib.get('name')
        print(image_name)
        if not os.path.exists(os.path.join(path, 'images', image_name)): # 이미지가 존재하지 않을 경우 pass
            continue
        image_witdh = image_attrib.get('width')
        image_height = image_attrib.get('height')
        polylines = image.findall('polyline') # 있음 라벨링
        polygons = image.findall('polygon') # 없음 라벨링
        
        for polyline in polylines: # 있음
            polyline_label, width, bb_points = ut.get_occupied_info(polyline, image_witdh, image_height)
            print(polyline_label) 
            if width > 250: # 길이가 너무 짧은 경우 넘어감
                occupied_cnt += 1 
                datasets['occupied'][str(occupied_cnt)]={'label':polyline_label, 'src':image_name,
                                                         'dataset':dataset_name,
                                                         'frame':image_frame,
                                                         'width': width,
                                                         'bb_points':bb_points
                                                        }
            
        for polygon in polygons:
            label = polygon.get('label')
            if label == '없음': # 없음
                empty_label, width, bb_points, poly_points, empty_type = ut.get_empty_info(polygon, image_witdh, image_height)
                print(empty_label, empty_type)
                if (type(bb_points) != None) and (width > 250): # point 정보가 없거나 길이가 너무 짧은 경우 넘어감
                    empty_cnt += 1
                    datasets['empty'][str(empty_cnt)]={'label':empty_label, 'src':image_name,
                                                       'dataset':dataset_name,
                                                       'frame':image_frame,
                                                       'width':width,
                                                       'type':empty_type,
                                                       'bb_points':bb_points,
                                                       'poly_points':poly_points
                                                      }
                
            elif label == '장애물': # 장애물
                obs_label, width, bb_points, poly_points, obs_type = ut.get_obstacle_info(polygon, image_witdh, image_height)
                print(obs_label, obs_type)
                obstacle_cnt += 1
                datasets['obstacle'][str(obstacle_cnt)]={'label':obs_label, 'src':image_name,
                                                         'dataset':dataset_name,
                                                         'frame':image_frame,
                                                         'width': width,
                                                         'type':obs_type,
                                                         'poly_points':poly_points
                                                        }
                
        #occupied_cnt += len(polylines)
        #empty_cnt += len(polygons)
        #print(image_name, '| 있음', len(polylines), '없음', len(polygons))
    

annotations.xml
extend1.jpg
4
bb [ 167.35 1809.4 ] [1361.8 1802. ]
ps: [[ 167.35       1928.84729225]
 [1361.8        1921.44729225]
 [1361.8        1125.1320106 ]
 [ 167.35       1132.5320106 ]] 1194.4729224641303 [[1030.6  1247.54]
 [ 167.35 1809.4 ]
 [1361.8  1802.  ]
 [1559.   1232.7 ]]
없음 1
extend10.jpg
4
bb [ 169.35 1820.21] [1363.8  1812.81]
ps: [[ 169.35       1939.65729225]
 [1363.8        1932.25729225]
 [1363.8        1135.9420106 ]
 [ 169.35       1143.3420106 ]] 1194.4729224641303 [[1032.61 1258.35]
 [ 169.35 1820.21]
 [1363.8  1812.81]
 [1561.   1243.51]]
없음 1
extend100.jpg
4
bb [1753.93 1603.11] [2530.22 1626.81]
ps: [[1753.93       1680.77516942]
 [2530.22       1704.47516942]
 [2530.22       1186.70737329]
 [1753.93       1163.00737329]] 776.6516941975982 [[1839.85 1203.11]
 [1753.93 1603.11]
 [2530.22 1626.81]
 [2198.37 1217.93]]
없음 1
4
bb [2565.78 1626.81] [3339.11 1656.44]
ps: [[2565.78       1704.19974259]
 [3339.11       1733.82974259]
 [3339.11       1217.8981253

4
bb [ 317.43 1875.5 ] [1492.05 1744.42]
ps: [[ 317.43       1993.6911211 ]
 [1492.05       1862.6111211 ]
 [1492.05       1074.67031374]
 [ 317.43       1205.75031374]] 1181.911211047598 [[ 911.21 1394.86]
 [ 317.43 1875.5 ]
 [1492.05 1744.42]
 [1474.06 1358.88]]
없음 1
extend1444.jpg
4
bb [ 317.43 1875.5 ] [1492.05 1744.42]
ps: [[ 317.43       1993.6911211 ]
 [1492.05       1862.6111211 ]
 [1492.05       1074.67031374]
 [ 317.43       1205.75031374]] 1181.911211047598 [[ 911.21 1394.86]
 [ 317.43 1875.5 ]
 [1492.05 1744.42]
 [1474.06 1358.88]]
없음 1
extend1445.jpg
4
bb [ 317.43 1875.5 ] [1492.05 1744.42]
ps: [[ 317.43       1993.6911211 ]
 [1492.05       1862.6111211 ]
 [1492.05       1074.67031374]
 [ 317.43       1205.75031374]] 1181.911211047598 [[ 911.21 1394.86]
 [ 317.43 1875.5 ]
 [1492.05 1744.42]
 [1474.06 1358.88]]
없음 1
extend1446.jpg
4
bb [ 317.43 1875.5 ] [1492.05 1744.42]
ps: [[ 317.43       1993.6911211 ]
 [1492.05       1862.6111211 ]
 [1492.05       1074.67031374]
 [ 317.

4
bb [ 365.78 1805.15] [1434.07 1806.96]
ps: [[ 365.78       1911.97915333]
 [1434.07       1913.78915333]
 [1434.07       1201.59479778]
 [ 365.78       1199.78479778]] 1068.2915333372252 [[1266.6  1266.5 ]
 [ 365.78 1805.15]
 [1434.07 1806.96]
 [1708.87 1259.17]]
없음 2
4
bb [1486.84 1786.95] [2618.82 1770.57]
ps: [[1486.84       1900.15985049]
 [2618.82       1883.77985049]
 [2618.82       1129.04751389]
 [1486.84       1145.42751389]] 1132.0985049014068 [[1734.35 1255.53]
 [1486.84 1786.95]
 [2618.82 1770.57]
 [2160.21 1248.25]]
없음 5
extend1767.jpg
4
bb [1486.84 1786.95] [2618.82 1770.57]
ps: [[1486.84       1900.15985049]
 [2618.82       1883.77985049]
 [2618.82       1129.04751389]
 [1486.84       1145.42751389]] 1132.0985049014068 [[1734.35 1255.53]
 [1486.84 1786.95]
 [2618.82 1770.57]
 [2160.21 1248.25]]
없음 5
4
bb [ 365.78 1805.15] [1434.07 1806.96]
ps: [[ 365.78       1911.97915333]
 [1434.07       1913.78915333]
 [1434.07       1201.59479778]
 [ 365.78       1199.78479778]] 10

4
bb [ 480.   1655.53] [1526.2 1676.5]
ps: [[ 480.         1760.17101399]
 [1526.2        1781.14101399]
 [1526.2        1083.53425405]
 [ 480.         1062.56425405]] 1046.4101399069104 [[1225.63 1278.06]
 [ 480.   1655.53]
 [1526.2  1676.5 ]
 [1701.   1282.7 ]]
없음 3
4
bb [2863.69 1667.18] [3781.75 1662.52]
ps: [[2863.69       1758.98718268]
 [3781.75       1754.32718268]
 [3781.75       1142.27929814]
 [2863.69       1146.93929814]] 918.0718268196666 [[2358.1  1313.  ]
 [2863.69 1667.18]
 [3781.75 1662.52]
 [2775.15 1294.37]]
없음 3
extend2219.jpg
4
bb [ 480.   1655.53] [1526.2 1676.5]
ps: [[ 480.         1760.17101399]
 [1526.2        1781.14101399]
 [1526.2        1083.53425405]
 [ 480.         1062.56425405]] 1046.4101399069104 [[1225.63 1278.06]
 [ 480.   1655.53]
 [1526.2  1676.5 ]
 [1701.   1282.7 ]]
없음 3
4
bb [2863.69 1667.18] [3781.75 1662.52]
ps: [[2863.69       1758.98718268]
 [3781.75       1754.32718268]
 [3781.75       1142.27929814]
 [2863.69       1146.93929814]] 918.071

4
bb [ 233.85 1230.91] [1106.83 1425.46]
ps: [[ 233.85       1320.34957641]
 [1106.83       1514.89957641]
 [1106.83        918.63573366]
 [ 233.85        724.08573366]] 894.3957641335293 [[1301.4   986.5 ]
 [ 233.85 1230.91]
 [1106.83 1425.46]
 [1775.3  1031.4 ]]
없음 4
4
bb [2153.9  1611.69] [3602.14 1885.45]
ps: [[2153.9        1759.07872532]
 [3602.14       2032.83872532]
 [3602.14       1050.24722318]
 [2153.9         776.48722318]] 1473.8872532185082 [[2284.6  1124.24]
 [2839.17 1191.35]
 [3602.14 1885.45]
 [2153.9  1611.69]]
없음 4
4
bb [1136.76 1425.46] [2159.39 1634.98]
ps: [[1136.76       1529.84729555]
 [2159.39       1739.36729555]
 [2159.39       1043.45199186]
 [1136.76        833.93199186]] 1043.8729555362568 [[1787.75 1041.35]
 [1136.76 1425.46]
 [2159.39 1634.98]
 [2289.09 1116.18]]
없음 4
extend2569.jpg
4
bb [ 233.85 1230.91] [1106.83 1425.46]
ps: [[ 233.85       1320.34957641]
 [1106.83       1514.89957641]
 [1106.83        918.63573366]
 [ 233.85        724.08573366]] 894

4
bb [1090.68  977.56] [1512.79  829.2 ]
ps: [[1090.68       1022.30232244]
 [1512.79        873.94232244]
 [1512.79        575.66017286]
 [1090.68        724.02017286]] 447.42322436368886 [[ 886.69  590.77]
 [1087.15  577.52]
 [1512.79  829.2 ]
 [1090.68  977.56]]
없음 5
4
bb [ 397.47 1015.62] [1090.68  977.56]
ps: [[ 397.47       1085.04540369]
 [1090.68       1046.98540369]
 [1090.68        584.14937911]
 [ 397.47        622.20937911]] 694.2540368625882 [[ 886.69  590.77]
 [ 663.27  600.57]
 [ 397.47 1015.62]
 [1090.68  977.56]]
없음 5
4
bb [1506.61  825.67] [1707.95  729.41]
ps: [[1506.61        847.98676123]
 [1707.95        751.72676123]
 [1707.95        602.948353  ]
 [1506.61        699.208353  ]] 223.16761234551947 [[1506.61  825.67]
 [1707.95  729.41]
 [1231.09  560.75]
 [1087.15  577.52]]
없음 5
extend2815.jpg
4
bb [1506.61  825.67] [1707.95  729.41]
ps: [[1506.61        847.98676123]
 [1707.95        751.72676123]
 [1707.95        602.948353  ]
 [1506.61        699.208353  ]] 223

4
bb [1194.    879.54] [1678.81  853.04]
ps: [[1194.          928.09337126]
 [1678.81        901.59337126]
 [1678.81        577.90422951]
 [1194.          604.40422951]] 485.5337126297204 [[1108.34  622.56]
 [1293.79  618.15]
 [1678.81  853.04]
 [1194.    879.54]]
없음 2
extend3155.jpg
4
bb [1194.    879.54] [1678.81  853.04]
ps: [[1194.          928.09337126]
 [1678.81        901.59337126]
 [1678.81        577.90422951]
 [1194.          604.40422951]] 485.5337126297204 [[1108.34  622.56]
 [1293.79  618.15]
 [1678.81  853.04]
 [1194.    879.54]]
없음 2
extend3156.jpg
4
bb [1194.    879.54] [1678.81  853.04]
ps: [[1194.          928.09337126]
 [1678.81        901.59337126]
 [1678.81        577.90422951]
 [1194.          604.40422951]] 485.5337126297204 [[1108.34  622.56]
 [1293.79  618.15]
 [1678.81  853.04]
 [1194.    879.54]]
없음 2
extend3157.jpg
4
bb [1194.    879.54] [1678.81  853.04]
ps: [[1194.          928.09337126]
 [1678.81        901.59337126]
 [1678.81        577.90422951]
 [1194.

ps: [[   0.         1960.93426967]
 [ 911.59       1707.20426967]
 [ 911.59       1076.37580522]
 [   0.         1330.10580522]] 946.2426966693059 [[ 673.16 1338.82]
 [   0.   1443.83]
 [   0.   1866.31]
 [ 911.59 1612.58]]
없음 1
extend501.jpg
4
bb [ 911.59 1612.58] [1565.07 1414.77]
ps: [[ 911.59       1680.85627015]
 [1565.07       1483.04627015]
 [1565.07       1027.87113584]
 [ 911.59       1225.68113584]] 682.7627014563698 [[ 673.16 1338.82]
 [1164.15 1255.81]
 [1565.07 1414.77]
 [ 911.59 1612.58]]
없음 1
4
bb [   0.   1866.31] [ 911.59 1612.58]
ps: [[   0.         1960.93426967]
 [ 911.59       1707.20426967]
 [ 911.59       1076.37580522]
 [   0.         1330.10580522]] 946.2426966693059 [[ 673.16 1338.82]
 [   0.   1443.83]
 [   0.   1866.31]
 [ 911.59 1612.58]]
없음 1
extend502.jpg
4
bb [ 911.59 1612.58] [1565.07 1414.77]
ps: [[ 911.59       1680.85627015]
 [1565.07       1483.04627015]
 [1565.07       1027.87113584]
 [ 911.59       1225.68113584]] 682.7627014563698 [[ 673.16 1338.

4
bb [1293.08 1695.58] [2276.82 1677.92]
ps: [[1293.08       1793.96985025]
 [2276.82       1776.30985025]
 [2276.82       1120.37751526]
 [1293.08       1138.03751526]] 983.8985024889511 [[1621.58 1269.94]
 [2063.12 1261.11]
 [2276.82 1677.92]
 [1293.08 1695.58]]
없음 5
4
bb [ 184.64 1714.13] [1293.08 1695.58]
ps: [[ 184.64       1824.98952084]
 [1293.08       1806.43952084]
 [1293.08       1067.37604857]
 [ 184.64       1085.92604857]] 1108.595208405665 [[1293.08 1695.58]
 [ 184.64 1714.13]
 [1187.82 1265.53]
 [1621.58 1269.94]]
없음 5
4
bb [3029.2  1672.62] [3763.92 1658.5 ]
ps: [[3029.2        1746.1055668 ]
 [3763.92       1731.9855668 ]
 [3763.92       1242.08178813]
 [3029.2        1256.20178813]] 734.8556680056298 [[2891.44 1307.03]
 [2469.33 1315.86]
 [3029.2  1672.62]
 [3763.92 1658.5 ]]
없음 1
extend835.jpg
4
bb [3029.2  1672.62] [3763.92 1658.5 ]
ps: [[3029.2        1746.1055668 ]
 [3763.92       1731.9855668 ]
 [3763.92       1242.08178813]
 [3029.2        1256.20178813]] 734.85

In [24]:
datasets

{'occupied': {},
 'empty': {'1': {'label': '없음',
   'src': 'extend1.jpg',
   'dataset': 'Extend_Empty',
   'frame': '0',
   'width': 1194.4729224641303,
   'type': 1,
   'bb_points': array([[ 167.35      , 1928.84729225],
          [1361.8       , 1921.44729225],
          [1361.8       , 1125.1320106 ],
          [ 167.35      , 1132.5320106 ]]),
   'poly_points': array([[1030.6 , 1247.54],
          [ 167.35, 1809.4 ],
          [1361.8 , 1802.  ],
          [1559.  , 1232.7 ]])},
  '2': {'label': '없음',
   'src': 'extend10.jpg',
   'dataset': 'Extend_Empty',
   'frame': '1',
   'width': 1194.4729224641303,
   'type': 1,
   'bb_points': array([[ 169.35      , 1939.65729225],
          [1363.8       , 1932.25729225],
          [1363.8       , 1135.9420106 ],
          [ 169.35      , 1143.3420106 ]]),
   'poly_points': array([[1032.61, 1258.35],
          [ 169.35, 1820.21],
          [1363.8 , 1812.81],
          [1561.  , 1243.51]])},
  '3': {'label': '없음',
   'src': 'extend100.jpg',

### Patch 정보 확인

In [25]:
datasets['empty']['1152']

{'label': '없음',
 'src': 'extend1766.jpg',
 'dataset': 'Extend_Empty',
 'frame': '852',
 'width': 1068.2915333372252,
 'type': 2,
 'bb_points': array([[ 365.78      , 1911.97915333],
        [1434.07      , 1913.78915333],
        [1434.07      , 1201.59479778],
        [ 365.78      , 1199.78479778]]),
 'poly_points': array([[1266.6 , 1266.5 ],
        [ 365.78, 1805.15],
        [1434.07, 1806.96],
        [1708.87, 1259.17]])}

### 카테고리별 개수 확인

In [26]:
len(datasets['occupied']), len(datasets['empty']), len(datasets['obstacle'])

(0, 4568, 0)

In [27]:
len(datasets['occupied']) + len(datasets['empty']) + len(datasets['obstacle'])

4568

### 세부구획 개수

In [28]:
num_empty = len(datasets['empty'])
print("총 라벨링된 세부구획 개수: {}".format(num_empty))

총 라벨링된 세부구획 개수: 4568


In [29]:
num_empty_type_list = [0, 0, 0, 0, 0, 0] # 일반형, 경형, 장애인전용, 여성우선주차장, 환경친화적 자동차 전용, 기타
for key in datasets['empty'].keys(): # frame 1,2,3,...
    empty_type = datasets['empty'][key]['type'] # 세부구획 0 1 2 3 4 5
    num_empty_type_list[empty_type] += 1
print("세부구획 개수 : 일반형 {} 경형 {} 장애인전용 {} 여성우선주차장 {} 환경친화적 자동차 전용 {} 기타 {}".format(*num_empty_type_list))

세부구획 개수 : 일반형 30 경형 1114 장애인전용 1116 여성우선주차장 229 환경친화적 자동차 전용 606 기타 1473


### 장애물 종류 개수

In [30]:
num_obstacle = len(datasets['obstacle'])
print("총 라벨링된 장애물 개수: {}".format(num_obstacle))

총 라벨링된 장애물 개수: 0


In [31]:
num_obs_list = [0, 0, 0] # 일반형 이륜차 기타
for key in datasets['obstacle'].keys():
    obs_type = datasets['obstacle'][key]['type']
    num_obs_list[obs_type] += 1
print("장애물 종류 개수 : 사람 {} 이륜차 {} 기타 {}".format(*num_obs_list))

장애물 종류 개수 : 사람 0 이륜차 0 기타 0


### 각 주차장 데이터셋 당 개수

In [32]:
# 개수 정보를 저장할 딕셔너리 초기화
num_dataset = {}
for dataset in dataset_list:
    num_dataset[dataset] = {'occupied':0, 'empty':0, 'obstacle': 0}

# 각 데이터셋마다 데이터 수 누적
for category in datasets.keys(): # occupied empty obstacle
    for data in datasets[category].keys(): # frame 1,2,3,4...
        dataset = datasets[category][data]['dataset'] # 데이터셋 이름 ex '어바니엘 가산'
        num_dataset[dataset][category] += 1 # 데이터 수 카운트

In [33]:
num_dataset

{'Extend_Empty': {'occupied': 0, 'empty': 4568, 'obstacle': 0}}